<a href="https://colab.research.google.com/github/Kharinaev/RePlay/blob/rl_ddpg/cql_replay_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Offline RL + RecSys 
> *Kharinaev Artyom*  
> *under the leadership of Panov Alexander*  
> *AIRI summer school, 2022*  

Added [CQL](https://arxiv.org/abs/2006.04779) implementation from [d3rlpy](https://github.com/takuseno/d3rlpy/blob/master/d3rlpy/algos/cql.py) to [RePlay](https://github.com/sb-ai-lab/RePlay) by SB AI Lab  

RePlay repo forked [here](https://github.com/Kharinaev/RePlay/tree/rl_ddpg)  
Changes:
- Fixed [ddpg.py](https://github.com/Kharinaev/RePlay/blob/rl_ddpg/replay/models/ddpg.py) to be able to run
- Added [cql.py](https://github.com/Kharinaev/RePlay/blob/rl_ddpg/replay/models/cql.py)

In [1]:
!pip install --upgrade pip setuptools wheel
!pip install pandas --upgrade
!pip install implicit --upgrade
# !pip install git+https://github.com/Kharinaev/RePlay.git@rl_ddpg
# unable to import this way, problems with pyproject.toml
!pip install replay-rec
!wget https://raw.githubusercontent.com/Kharinaev/RePlay/rl_ddpg/replay/models/cql.py
!pip install pytorch_ranger
!pip install tensorboardX
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install d3rlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 31.9 MB/s 
     |████████████████████████████████| 1.2 MB 53.8 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 65.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 MB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 21.4 MB/s eta 0:00:00


### **Restart** the environment before running next cells

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
import tqdm
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from replay.data_preparator import DataPreparator
from replay.experiment import Experiment
from replay.metrics import HitRate, NDCG, MAP, MRR
from replay.models import ALSWrap, KNN, LightFMWrap, SLIM

from cql import *

K = 5
SEED = 0

import torch
use_gpu = torch.cuda.is_available()

### MovieLens 1m dataset

In [3]:
!wget https://raw.githubusercontent.com/sb-ai-lab/RePlay/rl_ddpg/experiments/data/ml1m_items.dat
!wget https://raw.githubusercontent.com/sb-ai-lab/RePlay/rl_ddpg/experiments/data/ml1m_ratings.dat
!wget https://raw.githubusercontent.com/sb-ai-lab/RePlay/rl_ddpg/experiments/data/ml1m_users.dat

--2022-07-20 11:50:00--  https://raw.githubusercontent.com/sb-ai-lab/RePlay/rl_ddpg/experiments/data/ml1m_items.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 163604 (160K) [text/plain]
Saving to: ‘ml1m_items.dat’

ml1m_items.dat      100%[===================>] 159.77K  --.-KB/s    in 0.003s  

2022-07-20 11:50:01 (58.8 MB/s) - ‘ml1m_items.dat’ saved [163604/163604]

--2022-07-20 11:50:01--  https://raw.githubusercontent.com/sb-ai-lab/RePlay/rl_ddpg/experiments/data/ml1m_ratings.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [4]:
items = pd.read_csv("ml1m_items.dat", sep="\t", names=['item_id', 'name', 'genre'])
df = pd.read_csv("ml1m_ratings.dat", sep="\t", names=["user_id", "item_id", "relevance", "timestamp"])
users = pd.read_csv("ml1m_users.dat", sep="\t", names=["user_id", "gender", "age", "occupation", "zip_code"])

### Baselines

In [5]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

preparator = DataPreparator()
train_sp, _, _ = preparator(train)
test_sp, _, _ = preparator(test)
test = test.rename(columns = {'user_id' : 'user_idx', 'item_id' : 'item_idx'})
train_sp.count(), test_sp.count()

(800167, 200042)

In [6]:
e_base = Experiment(test, {MAP(): K, NDCG(): K, HitRate(): K, MRR(): K})

baselines = {
  'ALS': ALSWrap(seed=SEED), 
  'KNN': KNN(num_neighbours=K), 
  'LightFM': LightFMWrap(random_state=SEED), 
  'SLIM': SLIM(seed=SEED)
}

In [7]:
for key in tqdm.auto.tqdm(baselines.keys(), desc='Model'):
  model = baselines[key]
  model.fit(log=train_sp)
  pred = model.predict(log = test_sp, k=K).toPandas()
  e_base.add_result(key, pred)

e_base.results

Model:   0%|          | 0/4 [00:00<?, ?it/s]

20-Jul-22 11:51:30, replay, WARNING: This model can't predict cold users, they will be ignored
20-Jul-22 11:51:30, replay, WARNING: This model can't predict cold items, they will be ignored
20-Jul-22 11:51:30, replay, WARNING: This model can't predict cold users, they will be ignored
20-Jul-22 11:51:30, replay, WARNING: This model can't predict cold items, they will be ignored
20-Jul-22 11:53:55, replay, WARNING: This model can't predict cold items, they will be ignored
20-Jul-22 11:53:55, replay, WARNING: This model can't predict cold items, they will be ignored
20-Jul-22 11:56:55, replay, WARNING: This model can't predict cold users, they will be ignored
20-Jul-22 11:56:55, replay, WARNING: This model can't predict cold items, they will be ignored
20-Jul-22 11:56:55, replay, WARNING: This model can't predict cold users, they will be ignored
20-Jul-22 11:56:55, replay, WARNING: This model can't predict cold items, they will be ignored
20-Jul-22 11:58:14, replay, WARNING: This model ca

,HitRate@5,MAP@5,MRR@5,NDCG@5
ALS,0.063928,0.006308,0.029549,0.013422
KNN,0.068400,0.007664,0.035544,0.015534
LightFM,0.051507,0.005164,0.023777,0.010924
SLIM,0.054323,0.005315,0.024870,0.011333


### CQL

In [8]:
e_cql = Experiment(test, {MAP(): K, NDCG(): K, HitRate(): K, MRR(): K})

n_runs = 5
for i in tqdm.auto.trange(n_runs):
  model = CQL(use_gpu=use_gpu, k=K, n_epochs=3)
  model.fit(log=train_sp)
  pred = model.predict(log=test_sp, k=K).toPandas()
  e_cql.add_result(f'CQL run {i}', pred)

e_cql.results

  0%|          | 0/5 [00:00<?, ?it/s]

20-Jul-22 12:00:24, replay, INFO: The model is neural network with non-distributed training


2022-07-20 12:00.35 [debug    ] RoundIterator is selected.
2022-07-20 12:00.35 [info     ] Directory is created at d3rlpy_logs/CQL_20220720120035
2022-07-20 12:00.35 [debug    ] Building models...
2022-07-20 12:00.43 [debug    ] Models have been built.
2022-07-20 12:00.43 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220720120035/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10.0, 'batch_size': 256, 'conservative_weight': 5.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rat

Epoch 1/3:   0%|          | 0/3125 [00:00<?, ?it/s]

2022-07-20 12:02.14 [info     ] CQL_20220720120035: epoch=1 step=3125 epoch=1 metrics={'time_sample_batch': 0.0007977039337158203, 'time_algorithm_update': 0.027718832550048828, 'temp_loss': -8.466208019542695, 'temp': 1.0743542681121827, 'alpha_loss': 0.32059093093676494, 'alpha': 0.9963784733581543, 'critic_loss': 3379.4185449204256, 'actor_loss': 191.08944788330078, 'time_step': 0.02880756187438965} step=3125
2022-07-20 12:02.14 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220720120035/model_3125.pt


...

2022-07-20 12:26.40 [info     ] CQL_20220720122205: epoch=3 step=9375 epoch=3 metrics={'time_sample_batch': 0.0007493074035644532, 'time_algorithm_update': 0.027661860275268554, 'temp_loss': -12.535439424095154, 'temp': 1.883900902442932, 'alpha_loss': 6.76906134115383, 'alpha': 0.5027898385715485, 'critic_loss': 183.44129998474122, 'actor_loss': 82.89440930419921, 'time_step': 0.028751477355957032} step=9375
2022-07-20 12:26.40 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220720122205/model_9375.pt


20-Jul-22 12:26:40, replay, WARNING: This model can't predict cold users, they will be ignored
20-Jul-22 12:26:40, replay, WARNING: This model can't predict cold items, they will be ignored
20-Jul-22 12:26:40, replay, WARNING: This model can't predict cold users, they will be ignored
20-Jul-22 12:26:40, replay, WARNING: This model can't predict cold items, they will be ignored


Users: 6038, items: 3683


User:   0%|          | 0/6038 [00:00<?, ?it/s]

,HitRate@5,MAP@5,MRR@5,NDCG@5
CQL run 0,0.071547,0.007629,0.035351,0.015745
CQL run 1,0.071878,0.008041,0.036563,0.016253
CQL run 2,0.070719,0.007742,0.034943,0.015793
CQL run 3,0.062935,0.006969,0.032461,0.014182
CQL run 4,0.072209,0.007946,0.035942,0.016215


In [9]:
res_cql_5 = e_cql.results.mean().to_frame().T.rename(index={0:f'CQL avg {n_runs}'})
res_cql_5

,HitRate@5,MAP@5,MRR@5,NDCG@5
CQL avg 5,0.069858,0.007665,0.035052,0.015637


### Results

In [10]:
pd.concat([e_base.results, res_cql_5]).sort_values('NDCG@5', ascending=False)

,HitRate@5,MAP@5,MRR@5,NDCG@5
CQL avg 5,0.069858,0.007665,0.035052,0.015637
KNN,0.068400,0.007664,0.035544,0.015534
ALS,0.063928,0.006308,0.029549,0.013422
SLIM,0.054323,0.005315,0.024870,0.011333
LightFM,0.051507,0.005164,0.023777,0.010924
